In [1]:
# /Users/teliov/TUD/Thesis/Medvice/Notebooks/data/04_06_new_data/data/split
# So we can use the *thesislib* package
import sys
import os

module_path = os.path.abspath("..")

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from thesislib.utils import pathutils
import json

In [3]:
op_dir = pathutils.get_data_file("05_27_nlice/nlice/data")

In [4]:
ai_med_data_file = pathutils.get_data_file('definitions/ai-med-data.json')
with open(ai_med_data_file) as fp:
    ai_med_data = json.load(fp)

In [5]:
# first we need to determine the groupings for every symptom that exists
symptom_combinations = {}
nlice_key_set = []
for value in ai_med_data.values():
    symptoms = value.get("symptoms")
    for symptom in symptoms.values():
        slug = symptom.get("slug")
        nlice = symptom.get("nlice", {})
        if slug not in symptom_combinations:
            symptom_combinations[slug] = {
                "nature": ["0-None"],
                "vas": ["0-None"],
                "duration": ["0-None"],
                "location": ["0-None"]
            }
        combo = symptom_combinations[slug]
        for nlice_key, nlice_values in nlice.items():
            nlice_list = list(nlice_values.keys())
            nlice_key_set.append(nlice_key)
            curr_list = combo.get(nlice_key, [])
            new_list = sorted(list(set(curr_list + nlice_list)))
            
            combo[nlice_key] = new_list
        symptom_combinations[slug] = combo

In [6]:
# some replacements
symptom_combinations["headache"]["nature"] = ["0-None", "aching", "stabbing"]
symptom_combinations["abdominal-pain"]["location"] = ["0-None", "llq", "luq", "rlq", "ruq"]

In [7]:
reduction_map = {
    "headache": {
        "thunderclap": "stabbing",
        "throbbing": "aching",
        "pressure": "aching"
    },
    "abdominal-pain": {
        "epigastric": "ruq",
        "left-abdomen": "llq",
        "right-abdomen": "rlq",
        "umbilical": "rlq",
        "upper-abdomen": "luq"
    }
}

In [8]:
# for each symptom, create a map of the possible combinations
# assumption is that only one nature is possible
import itertools

symptom_combination_encoding_map = {}
for key, value in symptom_combinations.items():
    nature = value.get("nature")
    vas = value.get("vas")
    duration = value.get("duration")
    location = value.get("location")
    
    combos = itertools.product(nature, vas, duration, location)
    combos = map(lambda item: ";".join(item).strip(), combos)
    combos = list(filter(lambda item: len(item) > 0, combos))
    
#     # since we can have multiple locations, we add them to the list as well
#     all_combos = []

#     for idx in range(len(location)):
#         selection = itertools.combinations(location, idx+1)
#         for item in selection:
#             for combo in combos:
#                 curr_combo = combo + ";" +  ";".join(item)
#                 all_combos.append(curr_combo)
    

    encoding = {}
    for idx, combo in enumerate(combos):
        encoding[combo] = idx+1
    symptom_combination_encoding_map[key] = encoding

In [11]:
nlice_symptoms = [key for key in symptom_combination_encoding_map if len(symptom_combination_encoding_map[key]) > 1 ]

In [12]:
from glob import glob
import re
import hashlib
def slugify_condition(condition_name):
    condition_name = condition_name.lower()
    condition_name = re.sub(r"\s+", "-", condition_name)
    condition_name = re.sub(r"'", "-", condition_name)
    condition_name = re.sub(r"\(", "", condition_name)
    condition_name = re.sub(r"\)", "", condition_name)
    return condition_name

def get_symptom_condition_map(module_dir):
    module_files = glob(os.path.join(module_dir, "*.json"))
    symptom_map = {}
    condition_map = {}
    for file in module_files:
        with open(file) as fp:
            module = json.load(fp)
        states = module.get("states")
        for state in states.values():
            if state.get("type") != "Symptom" and state.get("type") != "ConditionOnset":
                continue
            if state.get("type") == "ConditionOnset":
                code = state.get("codes")[0]
                condition_map[code["code"]] = slugify_condition(code.get("display"))
                continue
            symptom_code = state.get("symptom_code")
            slug = slugify_condition(symptom_code.get("display"))
            slug_hash  = hashlib.sha224(slug.encode("utf-8")).hexdigest()
            symptom_map[slug_hash] = slug
    return symptom_map, condition_map

In [13]:
nlice_module_dir = "/Users/teliov/TUD/symcat-to-synthea/output/module_ai_med_adv"
nlice_symptom_map, nlice_condition_map = get_symptom_condition_map(nlice_module_dir)

In [14]:
actual_symptom_map = {key: value for key, value in nlice_symptom_map.items() if "nlice" not in value}

In [15]:
reverse_map = {value: key for key, value in actual_symptom_map.items()}

In [16]:
nlice_regex = re.compile("(.*)\-nlice-(.*)-(.*)")
nlice_transformation_map = {}
for key, value in nlice_symptom_map.items():
    match = nlice_regex.match(value)
    if match is None:
        xform = {
            "symptom": value,
            "nlice":  None,
            "value": None
        }
    else:
        xform = {
            "symptom": match.group(1),
            "nlice": match.group(2),
            "value": match.group(3)
        }
    nlice_transformation_map[key] = xform

In [17]:
def tranform_symptoms(symptom_str, transformation_map, symptom_combination_encoding_map, reduction_map):
    symptom_list = symptom_str.split(";")
    symptoms = {}
    for item in symptom_list:
        transformed = transformation_map.get(item)
        name = transformed.get("symptom")
        if name not in symptoms:
            symptoms[name] = {
                "nature": "0-None",
                "vas": "0-None",
                "duration": "0-None",
                "location": "0-None"
            }
        nlice = transformed.get("nlice")
        nlice_value = transformed.get("value")
        if nlice is not None and nlice_value is not None:
            if name in reduction_map and nlice_value in reduction_map[name]:
                nlice_value = reduction_map[name][nlice_value]
            symptoms[name][nlice] = nlice_value
        
    transformed_symptoms = []
    for key, value in symptoms.items():
        ordered = [value.get(item) for item in ["nature", "vas", "duration", "location"]]
        ordered = ";".join(ordered)
        encoding = symptom_combination_encoding_map[key][ordered]
        symptom_hash = hashlib.sha224(key.encode("utf-8")).hexdigest()
        transformed_symptoms.append("|".join([symptom_hash, str(encoding)]))
    return ";".join(transformed_symptoms)

In [18]:
m = "b1feb680c25ab9e5d88d282638d14ab9cf597c264dad9d6b6558e8dc;ba54bddaf72dfbdff9fc5aeee4364a993978f5c0d395442a33395302"
v = tranform_symptoms(m, nlice_transformation_map, symptom_combination_encoding_map, reduction_map)

In [31]:
def get_hash(key):
    return hashlib.sha224(key.encode("utf-8")).hexdigest()

In [32]:
transformation_map_file = os.path.join(op_dir, "transformation_map.json")
encoding_map_file = os.path.join(op_dir, "encoding_map.json")
encoding_count_file = os.path.join(op_dir, "encoding_count.json")
reduction_map_file = os.path.join(op_dir, "reduction_map.json")
with open(transformation_map_file, "w") as fp:
    json.dump(nlice_transformation_map, fp, indent=4)
with open(encoding_map_file, "w") as fp:
    json.dump(symptom_combination_encoding_map, fp, indent=4)
with open(reduction_map_file, "w") as fp:
    json.dump(reduction_map, fp, indent=4)
with open(encoding_count_file, "w") as fp:
    encounding_count = {get_hash(key): len(symptom_combination_encoding_map[key]) for key in symptom_combination_encoding_map}
    json.dump(encounding_count, fp)

['headache', 'limb-weakness', 'abdominal-pain', 'fever']

In [22]:
nlice_symptoms

['headache', 'limb-weakness', 'abdominal-pain', 'fever']

In [23]:
exc = {key: reverse_map[key] for key in nlice_symptoms}

In [25]:
sorted_symptoms = sorted(actual_symptom_map.keys())

In [26]:
nlice_symptoms_idx_offset = [9, 12, 20, 25]
nlice_names = [actual_symptom_map[sorted_symptoms[idx-3]] for idx in nlice_symptoms_idx_offset]

In [27]:
nlice_names

['limb-weakness', 'headache', 'abdominal-pain', 'fever']

In [28]:
len(symptom_combination_encoding_map)

24

In [29]:
len(actual_symptom_map)

24